# Systems Security Lab 1: Setting Up

In this lab we will set up the Chipwhisperer Nano and do some exercises to get familiar with the environment and start thinking about topics of the course.

# Background
## Chipwhisperer Nano

For the labs we will be using the Chipwhisperer Nano (see a full description [here](https://chipwhisperer.readthedocs.io/en/latest/Starter%20Kits/ChipWhisperer-Nano.html)). The Nano is a small embedded device which supports power analysis, a highly relevant security topic which will be covered later in the course. The target chip is a STM32F303F4P6, which has 16kB of FLASH and 4kB of SRAM.

# Setting up the hardware

In [ ]:
import chipwhisperer as cw
import matplotlib.pyplot as plt
import numpy as np
import utils
import tqdm.notebook as tqdm

In [ ]:
SCOPETYPE = 'CWNANO'
PLATFORM = 'CWNANO'
SS_VER="SS_VER_2_1"

In [ ]:
scope = cw.scope()
target = cw.target(scope, cw.targets.SimpleSerial2)
prog = cw.programmers.STM32FProgrammer

In [ ]:
scope.default_setup()

# Checking that everything works using a simple program

In [ ]:
%%bash -s "$PLATFORM" "$SS_VER"
cd ../hw/simpleserial-base/
make PLATFORM=$1 CRYPTO_TARGET=NONE SS_VER=$2 -j

In [ ]:
cw.program_target(scope, prog, f"../hw/simpleserial-base/simpleserial-base-{PLATFORM}.hex")

In [ ]:
msg = bytearray([0]*16) #simpleserial uses bytearrays
target.simpleserial_write('p', msg)
print(target.simpleserial_read('r', 16))

Above should give ```CWbytearray(b'00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00')```

In [ ]:
target.simpleserial_write('k', msg)
print(target.simpleserial_wait_ack())

Above should give ```bytearray(b'\x00')```

In [ ]:
scope.arm()
target.simpleserial_write('p', msg)
## fill in the rest...
scope.capture()
trace = scope.get_last_trace()
plt.plot(trace)

Above should give you a plot containing a power trace of 5000 points

# Setting Up The Use-Case
The running example for this course will be a use-case which relies on sensors which need to be secure to some extent. To mimic this situation, we simulate weather data from the Chipwhisperer Nano. In this original implementation there has been some 'encryption' added to the sensor to secure it, but it is a very bad implementation: it is simply an XOR of the data with a static mask.

In [ ]:
%%bash -s "$PLATFORM"
cd ../hw/secure-sensor-v1
make PLATFORM=$1 CRYPTO_TARGET=NONE -j

In [ ]:
cw.program_target(scope, prog, f"../hw/secure-sensor-v1/secure-sensor-{PLATFORM}.hex")

## The sensor interface + useful commands

This section shows some useful commands and how you can interact with the sensor.

In [ ]:
# This command resets the target. 
# This can be useful if it crashes somehow, and it also resets the PRNG so you can use it to have consistently repeatable data
utils.reset_target(scope)

In [ ]:
# This command serves only as a sanity check: it should mirror the input bytearray
target.send_cmd(0xac, 1, bytearray([3, 5, 8, 10, 15, 80]))
target.simpleserial_read(cmd='m')

In [ ]:
# The main command: polling the sensor for data
utils.reset_target(scope)
target.send_cmd(0x01, 1, bytearray([]))
encrypted_pl = target.simpleserial_read(cmd='s')
print(encrypted_pl)
# _ = parse_sensor_data(encrypted_pl, print_data=True)

The command gives as output an array of 16 bytes. We provide you with some utility commands to parse them in python:

In [ ]:
# This command will print the sensor values if print_data==True
# And returns the parsed values as a tuple (device_id, temperature, humidity, pressure, wind_speed, wind_direction, timestamp)
utils.parse_sensor_data(encrypted_pl, print_data=True)

If the values above do not make much sense, that is because they are 'encrypted' and should be 'decrypted' first

In [ ]:
# Lastly, parsing individual fields

# This code below captures several polling events
utils.reset_target(scope)
N = 3
masked_data = np.zeros((N, 16), dtype=np.uint8)
for i in tqdm.trange(N):
    target.send_cmd(0x01, 1, bytearray([]))
    masked_data[i] = target.simpleserial_read(cmd='s')

print("Raw masked data:")
print(masked_data)

# This command allows you to parse multiple of these events together and obtain the relevant field, and also parse individual events:
print("\nMasked timestamp data:")
print(utils.get_field("timestamp", masked_data, True))

print("\nMasked temperature of the first polling event (Fixed-point representation + parsed value):")
print("Fixed-Point:", utils.get_field("temperature", masked_data[0], True))
print("Parsed:", utils.get_field("temperature", masked_data[0], False))

print("\nNote that this temperature is equal to the one printed above, since the target was reset before the commands were sent")

# Exercise 1: Breaking the current encryption

As mentioned, the current encryption is a very simple implementation. A random mask is generated once when the device is (re)started and used to mask every individual byte sent over the wire:

```C
void encrypt(uint8_t *data, uint8_t data_len) {
    for (uint8_t i = 0; i < data_len; i++) {
        data[i] ^= mask;
    }
}
```

Furthermore, you have the following knowledge: 
- the data captured by the sensor looks as follows (implementing fixed-point):
```C
typedef struct {
    uint16_t device_id;
    int16_t temperature;
    uint16_t humidity;
    uint16_t pressure;
    uint16_t wind_speed;
    uint16_t wind_direction;
    uint16_t noise;
    uint16_t timestamp;
} SensorData;
```
- The timestamp increments with 1 after each command. I.e. if your first sensor poll request has timestamp T, the subsequent one will have timestamp T+1.
- The mask is a single byte, shared across all the bytes of the full 16-byte payload.

Use the information above to brainstorm how you can figure out the mask used for the encryption, and implement that attack to subsequently decrypt all the data coming from the sensor.

# Exercise 2: Implementing a new encryption

One (of many) weaknesses in the previous encryption scheme is the fact that the mask is shared between all the bytes. This allows an attacker to figure out the mask based on a known pattern (e.g. the timestamp pattern) and decrypt everything. One minor improvement is to utilize a different mask for each byte of the payload. Implement this simple improvement. You can change the `secure-sensor.c` code which is located in `hw/secure-sensor-v2`

In [ ]:
%%bash -s "$PLATFORM"
cd ../hw/secure-sensor-v2
make PLATFORM=$1 CRYPTO_TARGET=NONE -j

In [ ]:
cw.program_target(scope, prog, f"../hw/secure-sensor-v2/secure-sensor-{PLATFORM}.hex")

# Exercise 3: Breaking the new encryption

With the new improvement, we can not simply undo the whole mask by figuring out one byte. However, there are still patterns that an attacker can utilize to launch an attack. Think about how an attacker may do this (for a field other than the previously-used timestamp) and implement the attack for that field.

# Exercise 4: Brainstorming a new encryption

The final exercise is open-ended. So far you have analyzed and attacked two implementations of an encryption scheme, showing that it is very vulnerable. The exercise now is to think at a high-level how you would ideally implement security on this sensor. This concerns how you would implement an improved encryption scheme, but also extends to other properties you think would be desirable in this use-case.

Iterate in your group: devise a new method to perform the encryption, then think of attacks that could be possible against such a scheme. Repeatedly go back-and-forth between these steps until you are satisfied.

# Run below before exiting: disconnect from the hardware

In [ ]:
scope.dis()
target.dis()